# Base Directory

For each database in the cluster there is a subdirectory within `PGDATA/base`, named after the database's `OID` in pg_database. 

This subdirectory is the default location for the database's files. In particular, its dedicated system catalogs are stored there.

## Relation Forks

### Description

Each table and index is stored in forks (files), named after the table or index's `filenode number` (which sometimes will be the same as the OID of the relation but can be changed over time)

The `filenode number` can be found in `pg_class.relfilenode`.

### Structure

Elements:
- main fork (no suffix)
- more data when reaching the limit (suffix .1, .2, ...)
- free space map (suffix: fsm) - For heap and indexes (except hash index) keep track of available space in the relation
- visibility map (suffix: vm) - For heap only, keep track of which pages contain only tuples that are known to be visible to all active transactions
- initialization fork (suffix: init) - an empty table or index of the appropriate type for un-logged tables (without WAL).

<img src="./helpers/relation-fork.png" alt="drawing" width="800"/>

## Hands On

```SQL
DROP TABLE IF EXISTS my_table;
CREATE TABLE my_table(
    ID INT,
    NAME VARCHAR(20)
);

INSERT INTO my_table VALUES (1,'2pac');

SELECT
    oid
    ,relname
    ,relfilenode
FROM pg_class
WHERE relname = 'my_table';

VACUUM my_table;
```

```SQL
SELECT pg_relation_filepath('my_table');
```

```BASH
# Look for my_table table physical layout
FULL_RELATION_FILE_NODE=$(psql -A -t -c "SELECT pg_relation_filepath('my_table');")
DATABASE_FILE_NODE=$(echo $FULL_RELATION_FILE_NODE | awk -F "/" '{print $2}')
RELATION_FILE_NODE=$(echo $FULL_RELATION_FILE_NODE | awk -F "/" '{print $3}')
ls $PGDATA/base/$DATABASE_FILE_NODE | grep $RELATION_FILE_NODE
```

```SQL
SELECT pg_relation_filepath('my_table');
TRUNCATE my_table;
SELECT pg_relation_filepath('my_table'); -- New file node
```